# Regresión Logística: Detección de SPAM

En este ejercicio se muestran los fundamentos de la Regresión Logística planteando uno de los primeros problemas que fueron solucionados mediante el uso de técnicas de Machine Learning: la detección de SPAM.

## Enunciado del ejercicio

Se propone la construcción de un sistema de aprendizaje automático capaz de predecir si un correo determinado se corresponde con un correo de SPAM o no, para ello, se utilizará el siguiente conjunto de datos:

##### [2007 TREC Public Spam Corpus](https://plg.uwaterloo.ca/cgi-bin/cgiwrap/gvcormac/foo07)
The corpus trec07p contains 75,419 messages:

    25220 ham
    50199 spam

These messages constitute all the messages delivered to a particular
server between these dates:

    Sun, 8 Apr 2007 13:07:21 -0400
    Fri, 6 Jul 2007 07:04:53 -0400

### 1. Funciones complementarias

In [50]:
# Esta clase facilita el preprocesamiento de correos electrónicos que poseen código HTML
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.fed = []

    def handle_data(self, d):
        self.fed.append(d)

    def get_data(self):
        return ''.join(self.fed)

In [51]:
# Esta función se encarga de elimar los tags HTML que se encuentren en el texto del correo electrónico
def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [53]:
import email
import string
import nltk

class Parser:

    def __init__(self):
        self.stemmer = nltk.PorterStemmer()
        self.stopwords = set(nltk.corpus.stopwords.words('english'))
        self.punctuation = list(string.punctuation)

    def parse(self, email_path):
        """Parse an email."""
        with open(email_path, errors='ignore') as e:
            msg = email.message_from_file(e)
        return None if not msg else self.get_email_content(msg)

    def get_email_content(self, msg):
        """Extract the email content."""
        subject = self.tokenize(msg['Subject']) if msg['Subject'] else []
        body = self.get_email_body(msg.get_payload(),
                                   msg.get_content_type())
        content_type = msg.get_content_type()
        # Returning the content of the email
        return {"subject": subject,
                "body": body,
                "content_type": content_type}

    def get_email_body(self, payload, content_type):
        """Extract the body of the email."""
        body = []
        if type(payload) is str and content_type == 'text/plain':
            return self.tokenize(payload)
        elif type(payload) is str and content_type == 'text/html':
            return self.tokenize(strip_tags(payload))
        elif type(payload) is list:
            for p in payload:
                body += self.get_email_body(p.get_payload(),
                                            p.get_content_type())
        return body

    def tokenize(self, text):
        """Transform a text string in tokens. Perform two main actions,
        clean the punctuation symbols and do stemming of the text."""
        for c in self.punctuation:
            text = text.replace(c, "")
        text = text.replace("\t", " ")
        text = text.replace("\n", " ")
        tokens = list(filter(None, text.split(" ")))
        # Stemming of the tokens
        return [self.stemmer.stem(w) for w in tokens if w not in self.stopwords]

##### Lectura de un correo en formato raw

In [54]:
inmail = open("C:\\Users\\A0032\\Documents\\DOCS.MIOS\\TEC\\IA II\\Practica2\\correos\\Email1.eml").read()
print(inmail)

Delivered-To: alegcelis28@gmail.com
Received: by 2002:a05:7108:6a8c:b0:3a2:bc5c:e58d with SMTP id by12csp1205294gdb;
        Sat, 27 Jul 2024 15:34:15 -0700 (PDT)
X-Google-Smtp-Source: AGHT+IGFDtm3xpSsJiFNQPRelkVXRzg7Jb3kN+0ry+/LFv/51EiWUlqS697/iq52W1px5gTUrb2Z
X-Received: by 2002:a05:6830:6181:b0:708:d84d:f62a with SMTP id 46e09a7af769-70940c8d883mr5812453a34.22.1722119655606;
        Sat, 27 Jul 2024 15:34:15 -0700 (PDT)
ARC-Seal: i=1; a=rsa-sha256; t=1722119655; cv=none;
        d=google.com; s=arc-20160816;
        b=V3o7OJvxN2Zq+EA7HRtae5Mh7Fl+Twt4DH9j8klkZdaa4bH66tQbAO6ZuAo8fAM1eC
         blewlAGy9/3FG79+ozQAFJM5lK2Yy0quUHgcbvB7aJ1Dd4oBGNYTR7s+cha7JeCMu3qw
         6BsFAwNMYl58ThvPt3WJCSjcmghP2tfI03s1ay8RqgNmhj9WrGcySiN+G+fHi/7k6MEh
         WDRU4muuwzJ0UuXtzpm1QTiT085fCTvZ6YMaN6X2z7izI5gRjMb0FuN8QciwOoXWykwT
         se2ljVgQqC2i6EuL5PBcq4SwFDNT87U7+0lnCRUEGzcWbsMdFK3hU0QMxxgWOQEqO5Tm
         ex+g==
ARC-Message-Signature: i=1; a=rsa-sha256; c=relaxed/relaxed; d=google.com; s=a

##### Parsing del correo electrónico

In [55]:
p = Parser()
p.parse("C:\\Users\\A0032\\Documents\\DOCS.MIOS\\TEC\\IA II\\Practica2\\correos\\Email1.eml")

{'subject': ['utf8bqwrpw7nzienmqujficwgsg9sysbeaw1vwq4g8jtsg'],
 'body': ['hsbc',
  'hola',
  'danna',
  'alejandra',
  'gonzalez',
  'celi',
  'con',
  'dimo®',
  'pued',
  'enviar',
  'recibir',
  'transferencia',
  'solo',
  'necesita',
  'tu',
  'celular',
  'el',
  'teléfono',
  'de',
  'tu',
  'contacto',
  'conoc',
  'cómo',
  'vinculart',
  'dimo®',
  'desd',
  'tu',
  'app',
  'hsbc',
  'méxico',
  'recuerda',
  'que',
  'para',
  'vinculart',
  'con',
  'dimo®',
  'tendrá',
  'que',
  'contar',
  'con',
  'tu',
  'app',
  'hsbc',
  'méxico',
  'ingresa',
  'descárgala',
  'desd',
  'tu',
  'tienda',
  'de',
  'aplicacion',
  'token',
  'móvil',
  'activo',
  'conoc',
  'cómo',
  'activarlo',
  'httpsclickmail1hsbccommxqseecbfc27280c83b64fabc2f6a52fc6858cbfc16c2f8081303cfc957d768bff17e53f5387cc4cc32d47679386bcc0f06f089c598108c500315d65e52375e71859',
  'aquí',
  'número',
  'de',
  'celular',
  'registrado',
  'actualízalo',
  'en',
  'minuto',
  'desd',
  'tu',
  'app',
  'con

##### Lectura del índice

In [56]:
index = open("C:\\Users\\A0032\\Documents\\DOCS.MIOS\\TEC\\IA II\\Practica2\\trec07p\\full\\index2").readlines()
index

['spam ../data/Email1.eml\n',
 'spam ../data/Email2.eml\n',
 'spam ../data/Email3.eml\n',
 'spam ../data/Email4.eml\n',
 'spam ../data/Email5.eml\n',
 'ham ../data/Email6.eml\n',
 'ham ../data/Email7.eml\n',
 'ham ../data/Email8.eml\n',
 'ham ../data/Email9.eml\n',
 'ham ../data/Email10.eml\n',
 'ham ../data/Email11.eml\n',
 'ham ../data/Email12.eml\n',
 'ham ../data/Email13.eml\n',
 'ham ../data/Email14.eml\n',
 'ham ../data/Email15.eml\n',
 'ham ../data/Email16.eml\n',
 'ham ../data/Email17.eml\n',
 'ham ../data/Email18.eml\n',
 'ham ../data/Email19.eml\n',
 'ham ../data/Email20.eml\n']

In [57]:
import os

DATASET_PATH = "C:\\Users\\A0032\\Documents\\DOCS.MIOS\\TEC\\IA II\\Practica2\\trec07p"

def parse_index(path_to_index, n_elements):
    ret_indexes = []
    index = open(path_to_index).readlines()
    for i in range(n_elements):
        mail = index[i].split(" ../")
        label = mail[0]
        path = mail[1][:-1]
        ret_indexes.append({"label":label, "email_path":os.path.join(DATASET_PATH, path)})
    return ret_indexes

In [58]:
def parse_email(index):
    p = Parser()
    pmail = p.parse(index["email_path"])
    return pmail, index["label"]

In [59]:
indexes = parse_index("C:\\Users\\A0032\\Documents\\DOCS.MIOS\\TEC\\IA II\\Practica2\\trec07p\\full\\index2", 20)
indexes

[{'label': 'spam',
  'email_path': 'C:\\Users\\A0032\\Documents\\DOCS.MIOS\\TEC\\IA II\\Practica2\\trec07p\\data/Email1.eml'},
 {'label': 'spam',
  'email_path': 'C:\\Users\\A0032\\Documents\\DOCS.MIOS\\TEC\\IA II\\Practica2\\trec07p\\data/Email2.eml'},
 {'label': 'spam',
  'email_path': 'C:\\Users\\A0032\\Documents\\DOCS.MIOS\\TEC\\IA II\\Practica2\\trec07p\\data/Email3.eml'},
 {'label': 'spam',
  'email_path': 'C:\\Users\\A0032\\Documents\\DOCS.MIOS\\TEC\\IA II\\Practica2\\trec07p\\data/Email4.eml'},
 {'label': 'spam',
  'email_path': 'C:\\Users\\A0032\\Documents\\DOCS.MIOS\\TEC\\IA II\\Practica2\\trec07p\\data/Email5.eml'},
 {'label': 'ham',
  'email_path': 'C:\\Users\\A0032\\Documents\\DOCS.MIOS\\TEC\\IA II\\Practica2\\trec07p\\data/Email6.eml'},
 {'label': 'ham',
  'email_path': 'C:\\Users\\A0032\\Documents\\DOCS.MIOS\\TEC\\IA II\\Practica2\\trec07p\\data/Email7.eml'},
 {'label': 'ham',
  'email_path': 'C:\\Users\\A0032\\Documents\\DOCS.MIOS\\TEC\\IA II\\Practica2\\trec07p\\data/E

### 2. Preprocesamiento de los datos del conjunto de datos

Con las funciones presentadas anteriormente se permite la lectura de los correos electrónicos de manera programática y el procesamiento de los mismos para eliminar aquellos componentes que no resultan de utilidad para la detección de correos de SPAM. Sin embargo, cada uno de los correos sigue estando representado por un diccionario de Python con una serie de palabras.

In [63]:
# Cargamos el índice y las etiquetas en memoria
index = parse_index("C:\\Users\\A0032\\Documents\\DOCS.MIOS\\TEC\\IA II\\Practica2\\trec07p\\full\\index2", 1)
index

[{'label': 'spam',
  'email_path': 'C:\\Users\\A0032\\Documents\\DOCS.MIOS\\TEC\\IA II\\Practica2\\trec07p\\correos/Email1.eml'}]

In [65]:
# Leemos el primer correo
import os

open(index[0]["email_path"]).read()

'Delivered-To: alegcelis28@gmail.com\nReceived: by 2002:a05:7108:6a8c:b0:3a2:bc5c:e58d with SMTP id by12csp1205294gdb;\n        Sat, 27 Jul 2024 15:34:15 -0700 (PDT)\nX-Google-Smtp-Source: AGHT+IGFDtm3xpSsJiFNQPRelkVXRzg7Jb3kN+0ry+/LFv/51EiWUlqS697/iq52W1px5gTUrb2Z\nX-Received: by 2002:a05:6830:6181:b0:708:d84d:f62a with SMTP id 46e09a7af769-70940c8d883mr5812453a34.22.1722119655606;\n        Sat, 27 Jul 2024 15:34:15 -0700 (PDT)\nARC-Seal: i=1; a=rsa-sha256; t=1722119655; cv=none;\n        d=google.com; s=arc-20160816;\n        b=V3o7OJvxN2Zq+EA7HRtae5Mh7Fl+Twt4DH9j8klkZdaa4bH66tQbAO6ZuAo8fAM1eC\n         blewlAGy9/3FG79+ozQAFJM5lK2Yy0quUHgcbvB7aJ1Dd4oBGNYTR7s+cha7JeCMu3qw\n         6BsFAwNMYl58ThvPt3WJCSjcmghP2tfI03s1ay8RqgNmhj9WrGcySiN+G+fHi/7k6MEh\n         WDRU4muuwzJ0UuXtzpm1QTiT085fCTvZ6YMaN6X2z7izI5gRjMb0FuN8QciwOoXWykwT\n         se2ljVgQqC2i6EuL5PBcq4SwFDNT87U7+0lnCRUEGzcWbsMdFK3hU0QMxxgWOQEqO5Tm\n         ex+g==\nARC-Message-Signature: i=1; a=rsa-sha256; c=relaxed/relaxed; d=

In [66]:
# Parseamos el primer correo
mail, label = parse_email(index[0])
print("El correo es:", label)
print(mail)

El correo es: spam
{'subject': ['utf8bqwrpw7nzienmqujficwgsg9sysbeaw1vwq4g8jtsg'], 'body': ['hsbc', 'hola', 'danna', 'alejandra', 'gonzalez', 'celi', 'con', 'dimo®', 'pued', 'enviar', 'recibir', 'transferencia', 'solo', 'necesita', 'tu', 'celular', 'el', 'teléfono', 'de', 'tu', 'contacto', 'conoc', 'cómo', 'vinculart', 'dimo®', 'desd', 'tu', 'app', 'hsbc', 'méxico', 'recuerda', 'que', 'para', 'vinculart', 'con', 'dimo®', 'tendrá', 'que', 'contar', 'con', 'tu', 'app', 'hsbc', 'méxico', 'ingresa', 'descárgala', 'desd', 'tu', 'tienda', 'de', 'aplicacion', 'token', 'móvil', 'activo', 'conoc', 'cómo', 'activarlo', 'httpsclickmail1hsbccommxqseecbfc27280c83b64fabc2f6a52fc6858cbfc16c2f8081303cfc957d768bff17e53f5387cc4cc32d47679386bcc0f06f089c598108c500315d65e52375e71859', 'aquí', 'número', 'de', 'celular', 'registrado', 'actualízalo', 'en', 'minuto', 'desd', 'tu', 'app', 'conoc', 'cómo', 'httpsclickmail1hsbccommxqseecbfc27280c83b68413c3d935f69275a6dc7040be0dd6b2b090002333a980e253ae040d08120c15

El algoritmo de Regresión Logística no es capaz de ingerir texto como parte del conjunto de datos. Por lo tanto, deben aplicarse una serie de funciones adicionales que transformen el texto de los correos electrónicos parseados en una representación numérica.

##### Aplicación de CountVectorizer

In [67]:
from sklearn.feature_extraction.text import CountVectorizer

# Preapración del email en una cadena de texto
prep_email = [" ".join(mail['subject']) + " ".join(mail['body'])]

vectorizer = CountVectorizer()
X = vectorizer.fit(prep_email)

print("Email:", prep_email, "\n")
print("Características de entrada:", vectorizer.get_feature_names_out())

Email: ['utf8bqwrpw7nzienmqujficwgsg9sysbeaw1vwq4g8jtsghsbc hola danna alejandra gonzalez celi con dimo® pued enviar recibir transferencia solo necesita tu celular el teléfono de tu contacto conoc cómo vinculart dimo® desd tu app hsbc méxico recuerda que para vinculart con dimo® tendrá que contar con tu app hsbc méxico ingresa descárgala desd tu tienda de aplicacion token móvil activo conoc cómo activarlo httpsclickmail1hsbccommxqseecbfc27280c83b64fabc2f6a52fc6858cbfc16c2f8081303cfc957d768bff17e53f5387cc4cc32d47679386bcc0f06f089c598108c500315d65e52375e71859 aquí número de celular registrado actualízalo en minuto desd tu app conoc cómo httpsclickmail1hsbccommxqseecbfc27280c83b68413c3d935f69275a6dc7040be0dd6b2b090002333a980e253ae040d08120c151b4569440837b791dd15d8c561b3c29034c9e5dadb009eff aquí cuenta hsbc conoc todo acerca de dimo® en nuestro nuevo sitio httpsclickmail1hsbccommxqseecbfc27280c83b6a7ed335002b3fdedbc88f1416f7a51a1df09bbcfeca3b5f7fd1c6373d1f7523df549899926afe2b73f91b67cc57f4

In [68]:
X = vectorizer.transform(prep_email)
print("\nValues:\n", X.toarray())


Values:
 [[  1   6   5   2  19   2   1   1   1   2   2   2   2   1   1   2   2   2
    8   6   4   2   4   2   3   1   2   1   1   2   1   7   2   2   2   4
    1   1   2   2   6   1   1   1   1   1   1  10   2   4   2   1   1   1
    1   1   2   2   2   2   1   1   1   1   1   2   6   2   1   2   2  24
    4   2   6   2  10   2   4   1   1   2   2   8   2   2   1   1   1   2
    1   1   2   3   3   1   1   1   1   1   1   2   2   2   1   1   1   1
    1   1   1   2  17   1   1   1   1   1   1   1   1   1   1   2   1   1
    1   2   1   1 138   1   1   5   1   2   1   1   1   2   1   1   2   1
    6   2   1   6   1   2   2   1   1   1   2   1   1   1   1   1   1   1
    4   1   1   2   2   2   4   4   2   1   1   2   1   2   2   1   1   1
    1   7   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
    1   1   1   1   1   1   1   1   1   1   1   1   2  42   2   2   2   2
    2  14   2   2   2   3   2   2   2   2   2   2   1   1   3   1   1   2
    1   1   1   1   1   2   

##### Aplicación de OneHotEncoding

In [69]:
from sklearn.preprocessing import OneHotEncoder

prep_email = [[w] for w in mail['subject'] + mail['body']]

enc = OneHotEncoder(handle_unknown='ignore')
X = enc.fit_transform(prep_email)

print("Features:\n", enc.get_feature_names_out())
print("\nValues:\n", X.toarray())

Features:
 ['x0_0' 'x0_10px' 'x0_15px' 'x0_1px' 'x0_2024' 'x0_22px' 'x0_31' 'x0_32px'
 'x0_333' 'x0_abuttontbodytrtdwidth276px' 'x0_acerca' 'x0_activarlo'
 'x0_activarlo\xa0aquí' 'x0_activo' 'x0_actualízalo' 'x0_adisplayblock'
 'x0_adisplaynon' 'x0_al' 'x0_alejandra' 'x0_aplicacion' 'x0_app'
 'x0_aquí' 'x0_auto' 'x0_bajo' 'x0_banca' 'x0_banco' 'x0_barcol'
 'x0_barcolbarcol' 'x0_bodyborderrightsolid' 'x0_bodycolor000'
 'x0_bodycolor333' 'x0_bodywrapperoverflowhidden' 'x0_bottomspacerel'
 'x0_buttonitembuttonspacerdisplayblock' 'x0_ccc' 'x0_cccborderleftsolid'
 'x0_celi' 'x0_celular'
 'x0_checklistcomponentitemscontainerdividerdisplaynon'
 'x0_checklistcomponentitemscontainerwidth100' 'x0_clic' 'x0_clic\xa0aquí'
 'x0_comision' 'x0_con'
 'x0_connectwithusheadingconnectwithusiconswrapperdisplayblock'
 'x0_connectwithusiconconnectwithusiconlinkwidth28px'
 'x0_connectwithusiconitemltrpadding0'
 'x0_connectwithusiconitemrtlpadding0'
 'x0_connectwithusiconswrapperltrfloatleft'
 'x0_connectwith

##### Funciones auxiliares para preprocesamiento del conjunto de datos

In [70]:
def create_prep_dataset(index_path, n_elements):
    X = []
    y = []
    indexes = parse_index(index_path, n_elements)
    for i in range(n_elements):
        print("\rParsing email: {0}".format(i+1), end='')
        mail, label = parse_email(indexes[i])
        X.append(" ".join(mail['subject']) + " ".join(mail['body']))
        y.append(label)
    return X, y

### 3. Entrenamiento del algoritmo 

In [72]:
# Leemos únicamente un subconjunto de 100 correos electrónicos
X_train, y_train = create_prep_dataset("C:\\Users\\A0032\\Documents\\DOCS.MIOS\\TEC\\IA II\\Practica2\\full\\index", 100)
X_train

Parsing email: 100

['gener ciali brand qualitido feel pressur perform rise occas tri viagra anxieti thing past back old self',
 'typo debianreadmhi ive updat gulu i check mirror it seem littl typo debianreadm file exampl httpgulususherbrookecadebianreadm ftpftpfrdebianorgdebianreadm test lenni access releas diststest the current test develop snapshot name etch packag test unstabl pass autom test propog releas etch replac lenni like readmehtml yan morin consult en logiciel libr yanmorinsavoirfairelinuxcom 5149941556 to unsubscrib email debianmirrorsrequestlistsdebianorg subject unsubscrib troubl contact listmasterlistsdebianorg',
 'authent viagramega authenticv i a g r a discount pricec i a l i s discount pricedo miss it click httpwwwmoujsjkhchumcom authent viagra mega authenticv i a g r a discount pricec i a l i s discount pricedo miss it click',
 'nice talk yahey billi realli fun go night talk said felt insecur manhood i notic toilet quit small area worri websit i tell secret weapon extra 3 inch trust g

##### Aplicamos la vectorización a los datos

In [73]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)

In [74]:
print(X_train.toarray())
print("\nFeatures:", len(vectorizer.get_feature_names_out()))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

Features: 4911


In [75]:
import pandas as pd

pd.DataFrame(X_train.toarray(), columns=[vectorizer.get_feature_names_out()])

,0000,000000,00085,002,003,00450,009,01,01000u,0107,...,ӧanz,ӭѯ,ԡšݡ淶,լһʽ,չҵϣ,سŵþʊʊݾѯ,ڶҵţ,㶫иï26,饻jwk,쵼ã
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
y_train

['spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'spam',
 'ham',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'spam',
 'ham',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'ham',
 'ham',
 'spam',
 'spam',
 'spam',
 'ham',
 'ham',
 'ham',
 'spam',
 'ham',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'ham',
 'spam',
 'ham',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam']

###### Entrenamiento del algoritmo de regresión logística con el conjunto de datos preprocesado

In [77]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

### 4. Predicción

##### Lectura de un conjunto de correos nuevos

In [82]:
# Leemos 150 correos de nuestro conjunto de datos y nos quedamos únicamente con los 50 últimos 
# Estos 50 correos electrónicos no se han utilizado para entrenar el algoritmo
X, y = create_prep_dataset("C:\\Users\\A0032\\Documents\\DOCS.MIOS\\TEC\\IA II\\Practica2\\full\\index2", 19)
X_test = X
y_test = y

Parsing email: 19

##### Preprocesamiento de los correos con el vectorizador creado anteriormente

In [83]:
X_test = vectorizer.transform(X_test)

##### Predicción del tipo de correo

In [84]:
y_pred = clf.predict(X_test)
y_pred

array(['spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam',
       'spam', 'ham', 'spam', 'ham', 'spam', 'spam', 'spam', 'ham', 'ham',
       'ham', 'ham'], dtype='<U4')

In [85]:
print("Predicción:\n", y_pred)
print("\nEtiquetas reales:\n", y_test)

Predicción:
 ['spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'ham'
 'spam' 'ham' 'spam' 'spam' 'spam' 'ham' 'ham' 'ham' 'ham']

Etiquetas reales:
 ['spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham']


##### Evaluación de los resultados

In [86]:
from sklearn.metrics import accuracy_score

print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_pred)))

Accuracy: 0.579


### 5. Aumentando el conjunto de datos